# After doing the modeling and exploring some of the images, it became clear to me that some of the images have more than type of clothing shown. Also, some appear to be mislabeled. 

In [683]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [684]:
df = pd.read_json('/Users/butler/Documents/Image-Classification-Indian-Clothing/data/raw/train_data.json', lines=True)

In [685]:
df.head()

,image_url,image_path,brand,product_title,class_label,color
0,https://m.media-amazon.com/images/I/81XKaSKvly...,images/train/0.jpeg,Womanista,Women's Georgette Saree with Blouse Piece (TKI...,saree,NaN
1,https://m.media-amazon.com/images/I/91dDMDg+QT...,images/train/1.jpeg,Yashika,Women Linen Printed Sarees Free Size (BEE),saree,NaN
2,https://m.media-amazon.com/images/I/71TJ9p46PV...,images/train/2.jpeg,Womanista,Organza Floral Print Saree with Blouse Piece(W...,saree,NaN
3,https://m.media-amazon.com/images/I/81Y+je7CEg...,images/train/3.jpeg,PERFECTBLUE,Women's Digital Cotton Linen Blend Saree with ...,saree,NaN
4,https://m.media-amazon.com/images/I/61B8o9Ulqp...,images/train/4.jpeg,VAIVIDHYAM,Designer Sarees Women's Banarasi Cotton Silk S...,saree,NaN


In [686]:
df.tail()

,image_url,image_path,brand,product_title,class_label,color
91161,https://rukminim1.flixcart.com/image/309/371/k...,images/train/91161.jpeg,AURELIA,Women Printed Pure Cotton Straight Kurta,women_kurta,Blue
91162,https://rukminim1.flixcart.com/image/309/371/k...,images/train/91162.jpeg,Prime clothing,Women Checkered Cotton Blend Straight Kurta,women_kurta,White
91163,https://rukminim1.flixcart.com/image/309/371/k...,images/train/91163.jpeg,Vrisa,Women Embroidered Georgette Straight Kurta,women_kurta,White
91164,https://rukminim1.flixcart.com/image/309/371/k...,images/train/91164.jpeg,HELLEN VELLEN,Women Printed Rayon A-line Kurta,women_kurta,Dark Blue
91165,https://rukminim1.flixcart.com/image/309/371/k...,images/train/91165.jpeg,NEW STYLISH,Women Printed Rayon Crepe Blend Straight Kurta,women_kurta,Dark Green


In [687]:
df.shape

(91166, 6)

In [688]:
df.class_label.unique()

array(['saree', 'blouse', 'dhoti_pants', 'dupattas', 'gowns', 'kurta_men',
       'leggings_and_salwars', 'lehenga', 'mojaris_men', 'mojaris_women',
       'nehru_jackets', 'palazzos', 'petticoats', 'sherwanis',
       'women_kurta'], dtype=object)

In [689]:
df_revised = df.copy()

In [690]:
df_revised.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91166 entries, 0 to 91165
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   image_url      91166 non-null  object
 1   image_path     91166 non-null  object
 2   brand          91166 non-null  object
 3   product_title  91166 non-null  object
 4   class_label    91166 non-null  object
 5   color          10002 non-null  object
dtypes: object(6)
memory usage: 4.2+ MB


In [691]:
# Resetting index in case the original indices are needed
df_revised = df_revised.reset_index(names='number')

In [692]:
df_revised.head()

,number,image_url,image_path,brand,product_title,class_label,color
0,0,https://m.media-amazon.com/images/I/81XKaSKvly...,images/train/0.jpeg,Womanista,Women's Georgette Saree with Blouse Piece (TKI...,saree,NaN
1,1,https://m.media-amazon.com/images/I/91dDMDg+QT...,images/train/1.jpeg,Yashika,Women Linen Printed Sarees Free Size (BEE),saree,NaN
2,2,https://m.media-amazon.com/images/I/71TJ9p46PV...,images/train/2.jpeg,Womanista,Organza Floral Print Saree with Blouse Piece(W...,saree,NaN
3,3,https://m.media-amazon.com/images/I/81Y+je7CEg...,images/train/3.jpeg,PERFECTBLUE,Women's Digital Cotton Linen Blend Saree with ...,saree,NaN
4,4,https://m.media-amazon.com/images/I/61B8o9Ulqp...,images/train/4.jpeg,VAIVIDHYAM,Designer Sarees Women's Banarasi Cotton Silk S...,saree,NaN


In [693]:
df_revised.tail()

,number,image_url,image_path,brand,product_title,class_label,color
91161,91161,https://rukminim1.flixcart.com/image/309/371/k...,images/train/91161.jpeg,AURELIA,Women Printed Pure Cotton Straight Kurta,women_kurta,Blue
91162,91162,https://rukminim1.flixcart.com/image/309/371/k...,images/train/91162.jpeg,Prime clothing,Women Checkered Cotton Blend Straight Kurta,women_kurta,White
91163,91163,https://rukminim1.flixcart.com/image/309/371/k...,images/train/91163.jpeg,Vrisa,Women Embroidered Georgette Straight Kurta,women_kurta,White
91164,91164,https://rukminim1.flixcart.com/image/309/371/k...,images/train/91164.jpeg,HELLEN VELLEN,Women Printed Rayon A-line Kurta,women_kurta,Dark Blue
91165,91165,https://rukminim1.flixcart.com/image/309/371/k...,images/train/91165.jpeg,NEW STYLISH,Women Printed Rayon Crepe Blend Straight Kurta,women_kurta,Dark Green


In [694]:
# Using column names with smallest part of clothing name
columns = ['kurta', 'women', 'men', 'dhoti', 'saree', 'sherwani', 'palazzo', 
           'mojari', 'nehru', 'legging', 'salwar', 'petticoat', 'blouse', 
           'dupatta', 'gown','lehenga']


In [695]:
# Adding these columns to the df with the values being true if the product title contains the column name
for column in columns:
    df_revised[str(column)] = df_revised.product_title.str.contains(str(column), flags=re.IGNORECASE)

In [696]:
# Checking for rows that have girl in the product title
df_revised['girl'] = df_revised.product_title.str.contains('girl', flags=re.IGNORECASE)

In [697]:
# Putting the rows that have girl in the product title into the women category
c1 = (df_revised.women == False)
c2 = (df_revised.girl == True)

df_revised.loc[c2 & c2, 'women'] = True

In [698]:
# Checking for rows that have woman in the product title
df_revised['woman'] = df_revised.product_title.str.contains('woman', flags=re.IGNORECASE)

In [699]:
# Putting the rows that have woman in the product title into the women category
c1 = (df_revised.women == False)
c2 = (df_revised.woman == True)

df_revised.loc[c2 & c2, 'women'] = True

In [700]:
# Checking for rows that have boy in the product title
df_revised['man'] = df_revised.product_title.str.contains('man', flags=re.IGNORECASE)

In [701]:
# Checking for rows that have man in the product title
df_revised['boy'] = df_revised.product_title.str.contains('boy', flags=re.IGNORECASE)

In [702]:
# Putting the rows that have man in the product title into the men category
c1 = (df_revised.men == False)
c2 = (df_revised.man == True)

df_revised.loc[c2 & c2, 'men'] = True

In [703]:
# Putting the rows that have boy in the product title into the men category
c1 = (df_revised.men == False)
c2 = (df_revised.boy == True)

df_revised.loc[c2 & c2, 'men'] = True

In [704]:
# No longer needed
df_revised.drop(['girl', 'woman', 'man', 'boy'], axis=1, inplace=True)
df_revised.head()

,number,image_url,image_path,brand,product_title,class_label,color,kurta,women,men,...,palazzo,mojari,nehru,legging,salwar,petticoat,blouse,dupatta,gown,lehenga
0,0,https://m.media-amazon.com/images/I/81XKaSKvly...,images/train/0.jpeg,Womanista,Women's Georgette Saree with Blouse Piece (TKI...,saree,NaN,False,True,True,...,False,False,False,False,False,False,True,False,False,False
1,1,https://m.media-amazon.com/images/I/91dDMDg+QT...,images/train/1.jpeg,Yashika,Women Linen Printed Sarees Free Size (BEE),saree,NaN,False,True,True,...,False,False,False,False,False,False,False,False,False,False
2,2,https://m.media-amazon.com/images/I/71TJ9p46PV...,images/train/2.jpeg,Womanista,Organza Floral Print Saree with Blouse Piece(W...,saree,NaN,False,False,False,...,False,False,False,False,False,False,True,False,False,False
3,3,https://m.media-amazon.com/images/I/81Y+je7CEg...,images/train/3.jpeg,PERFECTBLUE,Women's Digital Cotton Linen Blend Saree with ...,saree,NaN,False,True,True,...,False,False,False,False,False,False,True,False,False,False
4,4,https://m.media-amazon.com/images/I/61B8o9Ulqp...,images/train/4.jpeg,VAIVIDHYAM,Designer Sarees Women's Banarasi Cotton Silk S...,saree,NaN,False,True,True,...,False,False,False,False,False,False,True,False,False,False


In [705]:
df_revised.columns

Index(['number', 'image_url', 'image_path', 'brand', 'product_title',
       'class_label', 'color', 'kurta', 'women', 'men', 'dhoti', 'saree',
       'sherwani', 'palazzo', 'mojari', 'nehru', 'legging', 'salwar',
       'petticoat', 'blouse', 'dupatta', 'gown', 'lehenga'],
      dtype='object')

In [706]:
# If the columns entitled 'men' and 'women' are both true, changing the 'men' column to false
c1 = (df_revised['men'] == True) 
c2 = (df_revised['women'] == True)

df_revised.loc[c1 & c2, 'men'] = False

In [707]:
# Making a column for women's kurtas
df_revised['women_kurta'] = [False]* 91166

In [708]:
# Making a column for men's kurtas
df_revised['kurta_men'] = [False]* 91166

In [709]:
df_revised.head()

,number,image_url,image_path,brand,product_title,class_label,color,kurta,women,men,...,nehru,legging,salwar,petticoat,blouse,dupatta,gown,lehenga,women_kurta,kurta_men
0,0,https://m.media-amazon.com/images/I/81XKaSKvly...,images/train/0.jpeg,Womanista,Women's Georgette Saree with Blouse Piece (TKI...,saree,NaN,False,True,False,...,False,False,False,False,True,False,False,False,False,False
1,1,https://m.media-amazon.com/images/I/91dDMDg+QT...,images/train/1.jpeg,Yashika,Women Linen Printed Sarees Free Size (BEE),saree,NaN,False,True,False,...,False,False,False,False,False,False,False,False,False,False
2,2,https://m.media-amazon.com/images/I/71TJ9p46PV...,images/train/2.jpeg,Womanista,Organza Floral Print Saree with Blouse Piece(W...,saree,NaN,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,3,https://m.media-amazon.com/images/I/81Y+je7CEg...,images/train/3.jpeg,PERFECTBLUE,Women's Digital Cotton Linen Blend Saree with ...,saree,NaN,False,True,False,...,False,False,False,False,True,False,False,False,False,False
4,4,https://m.media-amazon.com/images/I/61B8o9Ulqp...,images/train/4.jpeg,VAIVIDHYAM,Designer Sarees Women's Banarasi Cotton Silk S...,saree,NaN,False,True,False,...,False,False,False,False,True,False,False,False,False,False


In [710]:
# Making women_kurta true and kurta false if kurta is true and women is true
c1 = (df_revised['kurta'] == True) 
c2 = (df_revised['women'] == True)

df_revised.loc[c1 & c2, 'women_kurta'] = True
df_revised.loc[c1 & c2, 'kurta'] = False

In [711]:
# Making kurta_men true and kurta false if kurta is true and men is true
c1 = (df_revised['kurta'] == True) 
c2 = (df_revised['men'] == True)

df_revised.loc[c1 & c2, 'kurta_men'] = True
df_revised.loc[c1 & c2, 'kurta'] = False

In [712]:
# Making kurta false when if kurta is true and kurta_men is the class label
c1 = (df_revised['kurta'] == True) 
c2 = (df_revised.class_label == 'kurta_men')

df_revised.loc[c1 & c2, 'kurta'] = False

In [713]:
# Making kurta false when if kurta is true and women_kurta is the class label
c1 = (df_revised['kurta'] == True) 
c2 = (df_revised.class_label == 'women_kurta')

df_revised.loc[c1 & c2, 'kurta'] = False

In [714]:
df_kurta2.shape

(138, 25)

### This leaves 138 rows where kurta is true, men and women are false, and the class label does not contain kurta

In [715]:
# Since only women wear dupattas, if the class_label is dupatta and kurta is true, is must
# be a women's kurta
c1 = (df_revised.class_label == 'dupattas')
c2 = (df_revised.kurta == True)

df_revised.loc[c1 & c2, 'women_kurta'] = True

In [716]:
# Making columns for mojaris_men, mojaris_women, and leggings_and_salwars
df_revised['mojaris_men'] = [False]*91166
df_revised['mojaris_women'] = [False]*91166
df_revised['leggings_and_salwars'] = [False]*91166

In [717]:
# mojaris_women column is true if mojaris and women are true
c1 = (df_revised['mojari'] == True) 
c2 = (df_revised['women'] == True)

df_revised.loc[c1 & c2, 'mojaris_women'] = True

In [718]:
# mojaris_men column is true if mojaris and men are true
c1 = (df_revised['mojari'] == True) 
c2 = (df_revised['men'] == True)

df_revised.loc[c1 & c2, 'mojaris_men'] = True

In [719]:
# Putting the columns legging and salwar together
c1 = (df_revised['legging'] == True) 
c2 = (df_revised['salwar'] == True)

df_revised.loc[c1 | c2, 'leggings_and_salwars'] = True

In [720]:
# Dropping the columns that are no longer needed
df_revised.drop(['men', 'women', 'legging','salwar','mojari'], axis=1, inplace=True)

In [721]:
df_revised.head()

,number,image_url,image_path,brand,product_title,class_label,color,kurta,dhoti,saree,...,petticoat,blouse,dupatta,gown,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
0,0,https://m.media-amazon.com/images/I/81XKaSKvly...,images/train/0.jpeg,Womanista,Women's Georgette Saree with Blouse Piece (TKI...,saree,NaN,False,False,True,...,False,True,False,False,False,False,False,False,False,False
1,1,https://m.media-amazon.com/images/I/91dDMDg+QT...,images/train/1.jpeg,Yashika,Women Linen Printed Sarees Free Size (BEE),saree,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False
2,2,https://m.media-amazon.com/images/I/71TJ9p46PV...,images/train/2.jpeg,Womanista,Organza Floral Print Saree with Blouse Piece(W...,saree,NaN,False,False,True,...,False,True,False,False,False,False,False,False,False,False
3,3,https://m.media-amazon.com/images/I/81Y+je7CEg...,images/train/3.jpeg,PERFECTBLUE,Women's Digital Cotton Linen Blend Saree with ...,saree,NaN,False,False,True,...,False,True,False,False,False,False,False,False,False,False
4,4,https://m.media-amazon.com/images/I/61B8o9Ulqp...,images/train/4.jpeg,VAIVIDHYAM,Designer Sarees Women's Banarasi Cotton Silk S...,saree,NaN,False,False,True,...,False,True,False,False,False,False,False,False,False,False


In [722]:
# Removing the blouse label from all sarees in class_label because 
# models wearing sarees cannot be shown without a saree blouse

c1 = (df_revised['class_label'] == 'saree') 
c2 = (df_revised['blouse'] == True)

df_revised.loc[c1 & c2, 'blouse'] = False

In [723]:
# Removing the saree label from all blouses in class_label because 
# blouses are blouses for sarees or lehengas. Therefore, it is likely that saree 
# will be in the product description

c1 = (df_revised['class_label'] == 'blouse') 
c2 = (df_revised['saree'] == True)

df_revised.loc[c1 & c2, 'saree'] = False

In [724]:
# Removing the saree label from all blouses in class_label because 
# blouses are blouses for sarees or lehengas.Therefore, it is likely that lehenga 
# will be in the product description

c1 = (df_revised['class_label'] == 'blouse') 
c2 = (df_revised['lehenga'] == True)

df_revised.loc[c1 & c2, 'lehenga'] = False

In [725]:
# Removing the saree label from all petticoats in class_label because petticoats are worn under sarees. 
c1 = (df_revised['class_label'] == 'petticoat') 
c2 = (df_revised['saree'] == True)

df_revised.loc[c1 & c2, 'saree'] = False

In [726]:
df_revised.head()

,number,image_url,image_path,brand,product_title,class_label,color,kurta,dhoti,saree,...,petticoat,blouse,dupatta,gown,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
0,0,https://m.media-amazon.com/images/I/81XKaSKvly...,images/train/0.jpeg,Womanista,Women's Georgette Saree with Blouse Piece (TKI...,saree,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1,1,https://m.media-amazon.com/images/I/91dDMDg+QT...,images/train/1.jpeg,Yashika,Women Linen Printed Sarees Free Size (BEE),saree,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False
2,2,https://m.media-amazon.com/images/I/71TJ9p46PV...,images/train/2.jpeg,Womanista,Organza Floral Print Saree with Blouse Piece(W...,saree,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False
3,3,https://m.media-amazon.com/images/I/81Y+je7CEg...,images/train/3.jpeg,PERFECTBLUE,Women's Digital Cotton Linen Blend Saree with ...,saree,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False
4,4,https://m.media-amazon.com/images/I/61B8o9Ulqp...,images/train/4.jpeg,VAIVIDHYAM,Designer Sarees Women's Banarasi Cotton Silk S...,saree,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [727]:
df_revised.loc[0]

number                                                                  0
image_url               https://m.media-amazon.com/images/I/81XKaSKvly...
image_path                                            images/train/0.jpeg
brand                                                           Womanista
product_title           Women's Georgette Saree with Blouse Piece (TKI...
class_label                                                         saree
color                                                                 NaN
kurta                                                               False
dhoti                                                               False
saree                                                                True
sherwani                                                            False
palazzo                                                             False
nehru                                                               False
petticoat                             

In [728]:
# Renaming columns to conform to class labels
df_revised = df_revised.rename({'dhoti':'dhoti_pants', 'sherwani':'sherwanis', 'palazzo':'palazzos', 
             'nehru':'nehru_jackets', 'petticoat':'petticoats', 'dupatta':'dupattas', 
             'gown':'gowns'},axis=1)
df_revised.head()

,number,image_url,image_path,brand,product_title,class_label,color,kurta,dhoti_pants,saree,...,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
0,0,https://m.media-amazon.com/images/I/81XKaSKvly...,images/train/0.jpeg,Womanista,Women's Georgette Saree with Blouse Piece (TKI...,saree,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1,1,https://m.media-amazon.com/images/I/91dDMDg+QT...,images/train/1.jpeg,Yashika,Women Linen Printed Sarees Free Size (BEE),saree,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False
2,2,https://m.media-amazon.com/images/I/71TJ9p46PV...,images/train/2.jpeg,Womanista,Organza Floral Print Saree with Blouse Piece(W...,saree,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False
3,3,https://m.media-amazon.com/images/I/81Y+je7CEg...,images/train/3.jpeg,PERFECTBLUE,Women's Digital Cotton Linen Blend Saree with ...,saree,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False
4,4,https://m.media-amazon.com/images/I/61B8o9Ulqp...,images/train/4.jpeg,VAIVIDHYAM,Designer Sarees Women's Banarasi Cotton Silk S...,saree,NaN,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [729]:
# Changing each new column to false if the class_label is the same as the column name
columns = ['dhoti_pants', 'saree', 'sherwanis', 'palazzos', 'nehru_jackets', 'petticoats',
           'blouse','dupattas', 'gowns', 'lehenga', 'women_kurta', 'kurta_men', 
           'mojaris_men', 'mojaris_women', 'leggings_and_salwars']

for column in columns:
    c1 = (df_revised[str(column)] == True)
    c2 = (df_revised['class_label'] == str(column))
    df_revised.loc[c1 & c2, str(column)] = False

df_revised.head()

,number,image_url,image_path,brand,product_title,class_label,color,kurta,dhoti_pants,saree,...,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
0,0,https://m.media-amazon.com/images/I/81XKaSKvly...,images/train/0.jpeg,Womanista,Women's Georgette Saree with Blouse Piece (TKI...,saree,NaN,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,https://m.media-amazon.com/images/I/91dDMDg+QT...,images/train/1.jpeg,Yashika,Women Linen Printed Sarees Free Size (BEE),saree,NaN,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2,https://m.media-amazon.com/images/I/71TJ9p46PV...,images/train/2.jpeg,Womanista,Organza Floral Print Saree with Blouse Piece(W...,saree,NaN,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,3,https://m.media-amazon.com/images/I/81Y+je7CEg...,images/train/3.jpeg,PERFECTBLUE,Women's Digital Cotton Linen Blend Saree with ...,saree,NaN,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,4,https://m.media-amazon.com/images/I/61B8o9Ulqp...,images/train/4.jpeg,VAIVIDHYAM,Designer Sarees Women's Banarasi Cotton Silk S...,saree,NaN,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [730]:
columns = ['dhoti_pants', 'saree', 'kurta', 'sherwanis', 'palazzos', 'nehru_jackets', 'petticoats',
           'blouse','dupattas', 'gowns', 'lehenga', 'women_kurta', 'kurta_men', 
           'mojaris_men', 'mojaris_women', 'leggings_and_salwars']
# If a column is true, putting the column name as the value; otherwise, 0
def func(x):
    if x == True:
        return str(column)
    else:
        return 0

for column in columns:
    df_revised[str(column)] = df_revised[str(column)].apply(lambda x: func(x))

In [731]:
df_revised.head()

,number,image_url,image_path,brand,product_title,class_label,color,kurta,dhoti_pants,saree,...,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
0,0,https://m.media-amazon.com/images/I/81XKaSKvly...,images/train/0.jpeg,Womanista,Women's Georgette Saree with Blouse Piece (TKI...,saree,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,https://m.media-amazon.com/images/I/91dDMDg+QT...,images/train/1.jpeg,Yashika,Women Linen Printed Sarees Free Size (BEE),saree,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,https://m.media-amazon.com/images/I/71TJ9p46PV...,images/train/2.jpeg,Womanista,Organza Floral Print Saree with Blouse Piece(W...,saree,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,https://m.media-amazon.com/images/I/81Y+je7CEg...,images/train/3.jpeg,PERFECTBLUE,Women's Digital Cotton Linen Blend Saree with ...,saree,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,https://m.media-amazon.com/images/I/61B8o9Ulqp...,images/train/4.jpeg,VAIVIDHYAM,Designer Sarees Women's Banarasi Cotton Silk S...,saree,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [732]:
df_revised.tail()

,number,image_url,image_path,brand,product_title,class_label,color,kurta,dhoti_pants,saree,...,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
91161,91161,https://rukminim1.flixcart.com/image/309/371/k...,images/train/91161.jpeg,AURELIA,Women Printed Pure Cotton Straight Kurta,women_kurta,Blue,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91162,91162,https://rukminim1.flixcart.com/image/309/371/k...,images/train/91162.jpeg,Prime clothing,Women Checkered Cotton Blend Straight Kurta,women_kurta,White,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91163,91163,https://rukminim1.flixcart.com/image/309/371/k...,images/train/91163.jpeg,Vrisa,Women Embroidered Georgette Straight Kurta,women_kurta,White,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91164,91164,https://rukminim1.flixcart.com/image/309/371/k...,images/train/91164.jpeg,HELLEN VELLEN,Women Printed Rayon A-line Kurta,women_kurta,Dark Blue,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91165,91165,https://rukminim1.flixcart.com/image/309/371/k...,images/train/91165.jpeg,NEW STYLISH,Women Printed Rayon Crepe Blend Straight Kurta,women_kurta,Dark Green,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [733]:
# Number of gowns column that have gowns as value
df_revised.gowns.value_counts()

gowns
0        90982
gowns      184
Name: count, dtype: int64

In [734]:
# Gowns are sometimes called dresses
df_revised['dress'] = df_revised.product_title.str.contains('dress', flags=re.IGNORECASE)

In [735]:
# If dress is true and gowns is not (0), put the value gowns in from gowns column
c1 = (df_revised.gowns == 0)
c2 = (df_revised.dress == True)

df_revised.loc[c1 & c2, 'gowns'] = 'gowns'

df_revised.gowns.value_counts()

gowns
0        89187
gowns     1979
Name: count, dtype: int64

In [736]:
# Removing columns not needed
df_revised = df_revised.drop(['number', 'image_url', 'brand', 'color', 'dress'], axis=1)

### Making a dataframe for each label that is different than the class label and exploring the product titles and class labels of some of the rows

In [737]:
df_saree = df_revised[df_revised.saree == 'saree']
df_saree.shape

(1723, 19)

In [738]:
df_saree.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
20610,images/train/20610.jpeg,Jaipuri Fashion Women's Traditional Dhoti Pant...,dhoti_pants,0,0,saree,0,0,0,0,blouse,0,0,0,0,0,0,0,0
21180,images/train/21180.jpeg,Women's Designer Printed Rayon Dark Green Sare...,dhoti_pants,0,0,saree,0,0,0,0,0,0,0,0,0,0,0,0,0
21900,images/train/21900.jpeg,Blue Chanderi Pant Saree without Blouse (Fully...,dhoti_pants,0,0,saree,0,0,0,0,blouse,0,0,0,0,0,0,0,0
22385,images/train/22385.jpeg,Women's Woven Handloom Cotton Silk Saree with ...,dhoti_pants,0,0,saree,0,0,0,0,blouse,0,0,0,0,0,0,0,0
24793,images/train/24793.jpeg,Women's Cotton Dupatta Saree,dupattas,0,0,saree,0,0,0,0,0,0,0,0,0,0,0,0,0


In [739]:
df_saree.iloc[0,1], df_saree.iloc[0,2]

("Jaipuri Fashion Women's Traditional Dhoti Pant Ready to Wear Saree with Blouse (Ombre Green)",
 'dhoti_pants')

### Mislabeled:

In [740]:
df_saree.iloc[3,1], df_saree.iloc[3,2]

("Women's Woven Handloom Cotton Silk Saree with Blouse Piece (Multicolor)",
 'dhoti_pants')

In [741]:
df_blouse = df_revised[df_revised.blouse == 'blouse']
df_blouse.shape

(2337, 19)

In [742]:
df_blouse.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
20610,images/train/20610.jpeg,Jaipuri Fashion Women's Traditional Dhoti Pant...,dhoti_pants,0,0,saree,0,0,0,0,blouse,0,0,0,0,0,0,0,0
21900,images/train/21900.jpeg,Blue Chanderi Pant Saree without Blouse (Fully...,dhoti_pants,0,0,saree,0,0,0,0,blouse,0,0,0,0,0,0,0,0
22006,images/train/22006.jpeg,Womens Raglan Sleeve Body Blouse Nursing Top,dhoti_pants,0,0,0,0,0,0,0,blouse,0,0,0,0,0,0,0,0
22034,images/train/22034.jpeg,Womens Raglan Sleeve Body Blouse Top,dhoti_pants,0,0,0,0,0,0,0,blouse,0,0,0,0,0,0,0,0
22147,images/train/22147.jpeg,Girls Raglan Sleeve Body Blouse Top,dhoti_pants,0,0,0,0,0,0,0,blouse,0,0,0,0,0,0,0,0


In [743]:
df_blouse.tail()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
81355,images/train/81355.jpeg,Lucknow Chikankari Kurta Top Blouse Short Kurt...,women_kurta,0,0,0,0,0,0,0,blouse,0,0,0,0,0,0,0,0
82210,images/train/82210.jpeg,Lucknow Chikankari Kurta Top Blouse Short Kurt...,women_kurta,0,0,0,0,0,0,0,blouse,0,0,0,0,0,0,0,0
82918,images/train/82918.jpeg,Kota Doria Handmade Printed Cotton Saree with ...,women_kurta,0,0,saree,0,0,0,0,blouse,0,0,0,0,0,0,0,0
84674,images/train/84674.jpeg,Kota Doria Handmade Printed Cotton Saree with ...,women_kurta,0,0,saree,0,0,0,0,blouse,0,0,0,0,0,0,0,0
90157,images/train/90157.jpeg,by FBB Sequined Festive Blouse,women_kurta,0,0,0,0,0,0,0,blouse,0,0,0,0,0,0,0,0


In [744]:
df_blouse.iloc[1,1], df_blouse.iloc[1,2]

('Blue Chanderi Pant Saree without Blouse (Fully-Stitched)…', 'dhoti_pants')

### Mislabeled:

In [745]:
df_blouse.iloc[3,1], df_blouse.iloc[3,2]

('Womens Raglan Sleeve Body Blouse Top', 'dhoti_pants')

### Mislabeled:

In [746]:
df_blouse.iloc[4,1], df_blouse.iloc[4,2]

('Girls Raglan Sleeve Body Blouse Top', 'dhoti_pants')

In [747]:
df_dhoti_pants = df_revised[df_revised.dhoti_pants == 'dhoti_pants']
df_dhoti_pants.shape

(427, 19)

In [748]:
df_dhoti_pants.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
1277,images/train/1277.jpeg,Women's VISCOSE RAYON Dhoti Saree (MUSTARD YEL...,saree,0,dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2398,images/train/2398.jpeg,chios Women Cotton Dhoti Saree Without Blouse ...,saree,0,dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2598,images/train/2598.jpeg,Women's Viscose Rayon Dhoti Saree (Red),saree,0,dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27759,images/train/27759.jpeg,Women's Olive Green Poly Crepe Crop Top And Dh...,dupattas,0,dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,0,0,0
28621,images/train/28621.jpeg,Women's Pure Cotton Plain Semi Patiala Dhoti S...,dupattas,0,dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,0,0,leggings_and_salwars


In [749]:
df_dhoti_pants.iloc[0,1]

"Women's VISCOSE RAYON Dhoti Saree (MUSTARD YELLOW)"

In [750]:
df_dupattas = df_revised[df_revised.dupattas == 'dupattas']
df_dupattas.shape

(3666, 19)

In [751]:
df_dupattas.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
134,images/train/134.jpeg,Woman's kota silk dupatta,saree,0,0,0,0,0,0,0,0,dupattas,0,0,0,0,0,0,0
1622,images/train/1622.jpeg,Lami- 9 Meter Red Jari 3D Flower Patch Embroid...,saree,0,0,0,0,0,0,0,0,dupattas,0,lehenga,0,0,0,0,0
1623,images/train/1623.jpeg,Lami - 9 MTS. Gajri Resham Embroidery Border &...,saree,0,0,0,0,0,0,0,0,dupattas,0,lehenga,0,0,0,0,0
2455,images/train/2455.jpeg,Brocade Saree Falls LAce Border Reel Jacquard ...,saree,0,0,0,0,0,0,0,0,dupattas,0,lehenga,0,0,0,0,0
3023,images/train/3023.jpeg,Lami 9 Mtr Green Multicolor Flower Design Pars...,saree,0,0,0,0,0,0,0,0,dupattas,0,lehenga,0,0,0,0,0


### Mislabeled:

In [752]:
df_dupattas.iloc[0,1], df_dupattas.iloc[0,2]

("Woman's kota silk dupatta", 'saree')

In [753]:
df_gowns = df_revised[df_revised.gowns == 'gowns']
df_gowns.shape

(1979, 19)

In [754]:
df_gowns.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
1483,images/train/1483.jpeg,Rosegold Georgette Embellished Saree Style Dress,saree,0,0,0,0,0,0,0,0,0,gowns,0,0,0,0,0,0
2497,images/train/2497.jpeg,9 mtr pink color Laces Heavy Embroderi work re...,saree,0,0,0,0,0,0,0,0,0,gowns,0,0,0,0,0,0
2961,images/train/2961.jpeg,Generic Black Heavy Laces for Dresses Sarees D...,saree,0,0,0,0,0,0,0,0,0,gowns,0,0,0,0,0,0
3027,images/train/3027.jpeg,Women Regular Color Rayon Gown,saree,0,0,0,0,0,0,0,0,0,gowns,0,0,0,0,0,0
3829,images/train/3829.jpeg,Black Georgette Embellished Saree Style Dress,saree,0,0,0,0,0,0,0,0,0,gowns,0,0,0,0,0,0


In [755]:
df_gowns.iloc[1,1], df_gowns.iloc[1,2]

('9 mtr pink color Laces Heavy Embroderi work real mirror Border for Dresses, Sarees,',
 'saree')

### Mislabeled:

In [756]:
df_gowns.iloc[3,1], df_gowns.iloc[3,2]

('Women Regular Color Rayon Gown', 'saree')

In [757]:
df_kurta = df_revised[df_revised.kurta == 'kurta']
df_kurta.shape

(138, 19)

In [758]:
df_kurta.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
2014,images/train/2014.jpeg,Handloom Pure Cotton Jharna Khadi Saree Kurta ...,saree,kurta,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4799,images/train/4799.jpeg,Couple Set Cotton Silk saree with kurta combo,saree,kurta,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5246,images/train/5246.jpeg,Handloom Pure Cotton Saree Kurta Couple Set,saree,kurta,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5480,images/train/5480.jpeg,Handloom Pure Cotton Jharna Khadi Saree Kurta ...,saree,kurta,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6885,images/train/6885.jpeg,Handloom Pure Cotton Saree Kurta Couple Set,saree,kurta,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Mislabeled:

In [759]:
df_kurta.iloc[1,1], df_kurta.iloc[1,2]

('Couple Set Cotton Silk saree with kurta combo', 'saree')

In [760]:
df_kurta_men = df_revised[df_revised.kurta_men == 'kurta_men']
df_kurta_men.shape

(437, 19)

In [761]:
df_kurta_men.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
21949,images/train/21949.jpeg,NEW GEN Boys Full Sleev Jacket Kurta & DHUTI P...,dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,0,kurta_men,0,0,0
21967,images/train/21967.jpeg,Kids Ethnic Yellow Krishna Dress Dhoti and Kur...,dhoti_pants,0,0,0,0,0,0,0,0,0,gowns,0,0,kurta_men,0,0,0
22158,images/train/22158.jpeg,Boys' Black Cotton Blend Sherwani Style Kurta ...,dhoti_pants,0,0,0,sherwanis,0,0,0,0,0,0,0,0,kurta_men,0,0,0
22301,images/train/22301.jpeg,Kids Boys Festive & Party Full Sleeves Kurta &...,dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,0,kurta_men,0,0,0
22327,images/train/22327.jpeg,Men's Ethnic Poly Cotton Full Sleeve Checkered...,dhoti_pants,0,0,0,0,0,0,0,0,0,0,0,0,kurta_men,0,0,0


### Mislabeled:

In [762]:
df_kurta_men.iloc[0,1], df_kurta_men.iloc[0,2]

('NEW GEN Boys Full Sleev Jacket Kurta & DHUTI Pant (Orange_6-12M)',
 'dhoti_pants')

In [763]:
df_leggings_and_salwars = df_revised[df_revised.leggings_and_salwars == 'leggings_and_salwars']
df_leggings_and_salwars.shape

(2291, 19)

In [764]:
df_leggings_and_salwars.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
330,images/train/330.jpeg,Women's Fernandez Sequence Georgette Saree (Sa...,saree,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,leggings_and_salwars
2776,images/train/2776.jpeg,Women's Cotton Silk Blend Embroidered Salwar S...,saree,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,leggings_and_salwars
4629,images/train/4629.jpeg,Women's Italian Silk Crepe Teachers Saree Salw...,saree,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,leggings_and_salwars
4891,images/train/4891.jpeg,Women's Gold Sanna Silk With Embroidery Sequen...,saree,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,leggings_and_salwars
6033,images/train/6033.jpeg,Women's Cotton princess cut Salwar Suit Set,saree,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,leggings_and_salwars


### Correct labels:

In [765]:
df_leggings_and_salwars.iloc[0,1], df_leggings_and_salwars.iloc[0,2]

("Women's Fernandez Sequence Georgette Saree (Salwar_ER123150_Yellow)",
 'saree')

In [766]:
df_leggings_and_salwars.iloc[4,1]

"Women's Cotton princess cut Salwar Suit Set"

In [767]:
df_lehenga = df_revised[df_revised.lehenga == 'lehenga']
df_lehenga.shape

(312, 19)

In [768]:
df_lehenga.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
1622,images/train/1622.jpeg,Lami- 9 Meter Red Jari 3D Flower Patch Embroid...,saree,0,0,0,0,0,0,0,0,dupattas,0,lehenga,0,0,0,0,0
1623,images/train/1623.jpeg,Lami - 9 MTS. Gajri Resham Embroidery Border &...,saree,0,0,0,0,0,0,0,0,dupattas,0,lehenga,0,0,0,0,0
2235,images/train/2235.jpeg,Women's Silk Heavy Work Readymade Blouse for S...,saree,0,0,0,0,0,0,0,0,0,0,lehenga,0,0,0,0,0
2382,images/train/2382.jpeg,Women's Net Semi-stitched Lehenga Choli (67-IM...,saree,0,0,0,0,0,0,0,0,0,0,lehenga,0,0,0,0,0
2455,images/train/2455.jpeg,Brocade Saree Falls LAce Border Reel Jacquard ...,saree,0,0,0,0,0,0,0,0,dupattas,0,lehenga,0,0,0,0,0


### Mislabeled:

In [769]:
df_lehenga.iloc[3,1], df_lehenga.iloc[3,2]

("Women's Net Semi-stitched Lehenga Choli (67-IM5V-TOTL_Multicolored_Free Size)",
 'saree')

In [770]:
df_mojaris_men = df_revised[df_revised.mojaris_men == 'mojaris_men']
df_mojaris_men.shape

(102, 19)

In [771]:
df_mojaris_men.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
58976,images/train/58976.jpeg,Boy's Mojari,mojaris_women,0,0,0,0,0,0,0,0,0,0,0,0,0,mojaris_men,0,0
59077,images/train/59077.jpeg,Boys Designer Jutti/Mojari for Kids Ethnic Foo...,mojaris_women,0,0,0,0,0,0,0,0,0,0,0,0,0,mojaris_men,0,0
59141,images/train/59141.jpeg,Men's Mojari,mojaris_women,0,0,0,0,0,0,0,0,0,0,0,0,0,mojaris_men,0,0
59145,images/train/59145.jpeg,KADAM Men White Kolhapuri Design Leather Ethni...,mojaris_women,0,0,0,0,0,0,0,0,0,0,0,0,0,mojaris_men,0,0
59320,images/train/59320.jpeg,Men's Leather Mojaris,mojaris_women,0,0,0,0,0,0,0,0,0,0,0,0,0,mojaris_men,0,0


### Many of the images labeled mojaris_women are actually men's

In [772]:
df_mojaris_women = df_revised[df_revised.mojaris_women == 'mojaris_women']
df_mojaris_women.shape

(16, 19)

In [773]:
df_mojaris_women.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
56696,images/train/56696.jpeg,Ladies Jutti Women Jutti Women Mojari Tan,mojaris_men,0,0,0,0,0,0,0,0,0,0,0,0,0,0,mojaris_women,0
56738,images/train/56738.jpeg,Ladies Jutti Women Jutti Women Mojari Light Brown,mojaris_men,0,0,0,0,0,0,0,0,0,0,0,0,0,0,mojaris_women,0
56958,images/train/56958.jpeg,Womens Mojaris,mojaris_men,0,0,0,0,0,0,0,0,0,0,0,0,0,0,mojaris_women,0
57048,images/train/57048.jpeg,Men's/Women's Winter Woollen House Bedroom Ind...,mojaris_men,0,0,0,0,0,0,0,0,0,0,0,0,0,0,mojaris_women,0
57457,images/train/57457.jpeg,"Twinkle Star Juttis, Mojaris, Khussa Shoes for...",mojaris_men,0,0,0,0,0,0,0,0,0,0,0,0,0,0,mojaris_women,0


In [774]:
df_mojaris_women.iloc[4,1]

'Twinkle Star Juttis, Mojaris, Khussa Shoes for Women'

### Many of the images labeled mojaris_men are actually women's

In [775]:
df_nehru_jackets = df_revised[df_revised.nehru_jackets == 'nehru_jackets']
df_nehru_jackets.shape

(148, 19)

In [776]:
df_nehru_jackets.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
20012,images/train/20012.jpeg,Men's Nehru Jacket (VASMJ011_44_Multicolored_44),dhoti_pants,0,0,0,0,0,nehru_jackets,0,0,0,0,0,0,0,0,0,0
23003,images/train/23003.jpeg,Men's Kurta Pyjama & Printed Nehru Jacket set,dhoti_pants,0,0,0,0,0,nehru_jackets,0,0,0,0,0,0,kurta_men,0,0,0
36891,images/train/36891.jpeg,Printed Achkan Style Full Sleeves Kurta and Ch...,kurta_men,0,0,0,0,0,nehru_jackets,0,0,0,0,0,0,0,0,0,0
36898,images/train/36898.jpeg,Elegant Floral Print Sytlish Kurta and Churida...,kurta_men,0,0,0,0,0,nehru_jackets,0,0,0,0,0,0,0,0,0,0
37148,images/train/37148.jpeg,Printed Full Sleeves Kurta and Churidar Set wi...,kurta_men,0,0,0,0,0,nehru_jackets,0,0,0,0,0,0,0,0,0,0


### Mislabeled:

In [777]:
df_nehru_jackets.iloc[0,1], df_nehru_jackets.iloc[0,2]

("Men's Nehru Jacket (VASMJ011_44_Multicolored_44)", 'dhoti_pants')

In [778]:
df_palazzos = df_revised[df_revised.palazzos == 'palazzos']
df_palazzos.shape

(3145, 19)

In [779]:
df_palazzos.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
5974,images/train/5974.jpeg,HUNNY- BUNCH Premium Soft Uppada Silk Fabric U...,saree,0,0,0,0,palazzos,0,0,0,0,gowns,0,0,0,0,0,0
12412,images/train/12412.jpeg,"Omi Fashion Women's Ethnic Jacket, Blouse and ...",blouse,0,0,0,0,palazzos,0,0,0,0,0,0,0,0,0,0,0
19980,images/train/19980.jpeg,Women's Palazzo Wide Leg Bottom,dhoti_pants,0,0,0,0,palazzos,0,0,0,0,0,0,0,0,0,0,0
20016,images/train/20016.jpeg,Women's Palazzo Bottom,dhoti_pants,0,0,0,0,palazzos,0,0,0,0,0,0,0,0,0,0,0
20038,images/train/20038.jpeg,Women Rayon Flared Garara/Sharara Palazzo Pant...,dhoti_pants,0,0,0,0,palazzos,0,0,0,0,0,0,0,0,0,0,0


### Both mislabeled:

In [780]:
df_palazzos.iloc[2,1], df_palazzos.iloc[2,2]

("Women's Palazzo Wide Leg Bottom", 'dhoti_pants')

In [781]:
df_palazzos.iloc[2,1]

"Women's Palazzo Wide Leg Bottom"

In [782]:
df_petticoats = df_revised[df_revised.petticoats == 'petticoats']
df_petticoats.shape

(63, 19)

In [783]:
df_petticoats.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
227,images/train/227.jpeg,Women's Fine Knitted Sexy Inskirt Saree Pettic...,saree,0,0,0,0,0,0,petticoats,0,0,0,0,0,0,0,0,0
845,images/train/845.jpeg,Women's Yellow Shapewear Lycra Stretchable Sli...,saree,0,0,0,0,0,0,petticoats,0,0,0,0,0,0,0,0,0
871,images/train/871.jpeg,Women's Saree Shapewear Petticoat,saree,0,0,0,0,0,0,petticoats,0,0,0,0,0,0,0,0,0
879,images/train/879.jpeg,Women's Pure Cotton Readymade Inskirt Saree Pe...,saree,0,0,0,0,0,0,petticoats,0,0,0,0,0,0,0,0,0
890,images/train/890.jpeg,Women's Wine Stretchable Slim Fit Saree Shapew...,saree,0,0,0,0,0,0,petticoats,0,0,0,0,0,0,0,0,0


In [784]:
df_petticoats.iloc[0,1], df_petticoats.iloc[0,2]

("Women's Fine Knitted Sexy Inskirt Saree Petticoats with Lace (Free Size)",
 'saree')

### Both are mislabeled:

In [785]:
df_petticoats.iloc[2,1], df_petticoats.iloc[2,1], 

("Women's Saree Shapewear Petticoat", "Women's Saree Shapewear Petticoat")

In [786]:
df_sherwanis = df_revised[df_revised.sherwanis == 'sherwanis']
df_sherwanis.shape

(286, 19)

In [787]:
df_sherwanis.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
22158,images/train/22158.jpeg,Boys' Black Cotton Blend Sherwani Style Kurta ...,dhoti_pants,0,0,0,sherwanis,0,0,0,0,0,0,0,0,kurta_men,0,0,0
23016,images/train/23016.jpeg,Men's Velvet Harem Dhoti Pants for Sherwani & ...,dhoti_pants,0,0,0,sherwanis,0,0,0,0,0,0,0,0,0,0,0,0
23066,images/train/23066.jpeg,Men's Velvet Harem Dhoti Pants for Sherwani & ...,dhoti_pants,0,0,0,sherwanis,0,0,0,0,0,0,0,0,0,0,0,0
23210,images/train/23210.jpeg,Men's Velvet Ready to Wear Free Size Harem Dho...,dhoti_pants,0,0,0,sherwanis,0,0,0,0,0,0,0,0,kurta_men,0,0,0
23211,images/train/23211.jpeg,Men Brown Cotton Silk Sherwani and Cow Style D...,dhoti_pants,0,0,0,sherwanis,0,0,0,0,0,0,0,0,0,0,0,0


### Also mislabeled. It is a sherwani-style kurta, not a sherwani

In [788]:
df_sherwanis.iloc[0,1], df_sherwanis.iloc[0,2]

("Boys' Black Cotton Blend Sherwani Style Kurta Set(VASBSW120)", 'dhoti_pants')

In [789]:
df_women_kurta = df_revised[df_revised.women_kurta == 'women_kurta']
df_women_kurta.shape

(1711, 19)

In [790]:
df_women_kurta.head()

,image_path,product_title,class_label,kurta,dhoti_pants,saree,sherwanis,palazzos,nehru_jackets,petticoats,blouse,dupattas,gowns,lehenga,women_kurta,kurta_men,mojaris_men,mojaris_women,leggings_and_salwars
1112,images/train/1112.jpeg,Women's Khadi Handloom Cotton Mens & Womens Co...,saree,0,0,0,0,0,0,0,0,0,0,0,women_kurta,0,0,0,0
1865,images/train/1865.jpeg,Women's Crepe Straight Kurta,saree,0,0,0,0,0,0,0,0,0,0,0,women_kurta,0,0,0,0
2821,images/train/2821.jpeg,Women's Khadi Handloom Cotton Mens & Womens Co...,saree,0,0,0,0,0,0,0,0,0,0,0,women_kurta,0,0,0,0
3683,images/train/3683.jpeg,Women's Summer Cotton Mens & Womens Couple Set...,saree,0,0,0,0,0,0,0,0,0,0,0,women_kurta,0,0,0,0
4228,images/train/4228.jpeg,Women's Khadi Handloom Cotton Mens & Womens Co...,saree,0,0,0,0,0,0,0,0,0,0,0,women_kurta,0,0,0,0


### Mislabeled:

In [791]:
df_women_kurta.iloc[1,1], df_women_kurta.iloc[1,2]

("Women's Crepe Straight Kurta", 'saree')

In [792]:
df_revised.to_csv('/Users/butler/Documents/Image-Classification-Indian-Clothing/data/processed/train_other_categories.csv')

## While not all of the items in each dataframe are mislabeled, many are. 
## There are 1723 rows in the saree dataframe;
## 2337 rows in the blouse dataframe;
## 427 rows in the dhoti_pants dataframe;
## 3666 rows in the dupatta dataframe;
## 1979 rows in the gown dataframe;
## 138 rows in the kurta dataframe;
## 416 rows in the kurta_men dataframe;
## 2291 rows in the leggings_and_salwars dataframe;
## 2337 rows in the blouse dataframe;
## 312 rows in the lehenga dataframe;
## 96 rows in the mojaris_men dataframe;
## 13 rows in the mojaris_women dataframe;
## 148 rows in the Nehru_jackets dataframe;
## 3145 rows in the palazzos dataframe;
## 63 rows in the petticoats dataframe;
## 286 rows in the sherwani dataframe;
## and 1654 rows in the women_kurta dataframe.